In [3]:
import json
import os
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, LabelSet, CategoricalColorMapper
from bokeh.layouts import column
from bokeh.io import save

In [4]:
root = os.getcwd()

In [5]:
with open(os.path.join(root, 'jsons', 'stockRatiosTTM.json')) as f:
    ratios_ttm_json = json.load(f)

In [6]:
def make_comparison_scatter_plot(x_axis, y_axis):
    # x_axis and y_axis: tuple with 2 values, the first is the value in the json, the second is a pretified version
    x_list = []
    y_list = []
    names_list = []
    x_tickers = 0
    y_tickers = 0
    both_tickers = 0
    for ticker, data in ratios_ttm_json.items():
        data = data[0]
        if data[x_axis[0]]:
            x_tickers += 1
        if data[y_axis[0]]:
            y_tickers += 1
        if data[x_axis[0]] and data[y_axis[0]]:
            both_tickers += 1
            x_list.append(data[x_axis[0]])
            y_list.append(data[y_axis[0]])
            names_list.append(ticker)

    print(f"Companies with {x_axis[1]}: {x_tickers}")
    print(f"Companies with {y_axis[1]}: {y_tickers}")
    print(f"Companies with both {x_axis[1]} and {y_axis[1]}: {both_tickers}")
    source = ColumnDataSource(data={f"{x_axis[1]}": x_list,
                                    f"{y_axis[1]}": y_list,
                                    "Tickers": names_list})
    p = figure(title=f"{y_axis[1]} TTM vs {x_axis[1]} TTM of Prominent Gold Stocks")
    p.scatter(x=x_axis[1], y=y_axis[1], source=source)
    p.xaxis.axis_label = x_axis[1]
    p.yaxis.axis_label = y_axis[1]
    labels = LabelSet(x=x_axis[1], y=y_axis[1], text="Tickers", source=source, x_offset=5, y_offset=5)
    p.add_layout(labels)
    show(p)
make_comparison_scatter_plot(("quickRatioTTM", "Quick Ratio"), ('operatingProfitMarginTTM', "Operating Profit Margin"))

Companies with Quick Ratio: 17
Companies with Operating Profit Margin: 8
Companies with both Quick Ratio and Operating Profit Margin: 8


## Finding how many of each stats are available

In [18]:
ratios_ttm_json.pop('TXG.TO')

[{'dividendYielTTM': None,
  'dividendYielPercentageTTM': None,
  'peRatioTTM': 3.5590143,
  'pegRatioTTM': 0.06952493051162789,
  'payoutRatioTTM': None,
  'currentRatioTTM': 2.357718536887787,
  'quickRatioTTM': 1.494730316181029,
  'cashRatioTTM': 1.2151270923744575,
  'daysOfSalesOutstandingTTM': 17.415890816758356,
  'daysOfInventoryOutstandingTTM': 83.57507754059478,
  'operatingCycleTTM': 87.86940678308314,
  'daysOfPayablesOutstandingTTM': 70.05655902207626,
  'cashConversionCycleTTM': -32.03311483914385,
  'grossProfitMarginTTM': 0.4201227253491325,
  'operatingProfitMarginTTM': 0.3909225560727888,
  'pretaxProfitMarginTTM': 0.39060516292848074,
  'netProfitMarginTTM': 0.28343207786711805,
  'effectiveTaxRateTTM': 0.27437703141928493,
  'returnOnAssetsTTM': 0.2088563187027364,
  'returnOnEquityTTM': 0.27770291282263915,
  'returnOnCapitalEmployedTTM': 0.3294988407347958,
  'netIncomePerEBTTTM': 0.7256229685807151,
  'ebtPerEbitTTM': 0.9991880920162381,
  'ebitPerRevenueTTM': 0

In [16]:
ratios_dict = {}
for ticker, data in ratios_ttm_json.items():
    data = data[0]
    for key, item in data.items():
        if key not in ratios_dict:
            ratios_dict[key] = 0
        if item:
            ratios_dict[key] += 1
ratios_dict

{'dividendYielTTM': 3,
 'dividendYielPercentageTTM': 3,
 'peRatioTTM': 17,
 'pegRatioTTM': 17,
 'payoutRatioTTM': 2,
 'currentRatioTTM': 17,
 'quickRatioTTM': 17,
 'cashRatioTTM': 17,
 'daysOfSalesOutstandingTTM': 8,
 'daysOfInventoryOutstandingTTM': 5,
 'operatingCycleTTM': 6,
 'daysOfPayablesOutstandingTTM': 11,
 'cashConversionCycleTTM': 6,
 'grossProfitMarginTTM': 7,
 'operatingProfitMarginTTM': 8,
 'pretaxProfitMarginTTM': 9,
 'netProfitMarginTTM': 9,
 'effectiveTaxRateTTM': 9,
 'returnOnAssetsTTM': 18,
 'returnOnEquityTTM': 18,
 'returnOnCapitalEmployedTTM': 17,
 'netIncomePerEBTTTM': 18,
 'ebtPerEbitTTM': 17,
 'ebitPerRevenueTTM': 8,
 'debtRatioTTM': 18,
 'debtEquityRatioTTM': 18,
 'longTermDebtToCapitalizationTTM': 13,
 'totalDebtToCapitalizationTTM': 10,
 'interestCoverageTTM': 13,
 'cashFlowToDebtRatioTTM': 9,
 'companyEquityMultiplierTTM': 18,
 'receivablesTurnoverTTM': 8,
 'payablesTurnoverTTM': 11,
 'inventoryTurnoverTTM': 5,
 'fixedAssetTurnoverTTM': 7,
 'assetTurnoverTTM

In [20]:
width = 1700
height = 800
def make_bar_charts_for_ratios(ratios_list):
    """
    ratios_list is a list of tuples, the first one is the ratio name in the json, the second one is the clean name, the third one is true if a higher number is 'better'
    """
    figures_list = []
    for ratio in ratios_list:
        ratio_values_list = []
        ticker_list = []
        for ticker, data in ratios_ttm_json.items():
            data = data[0]
            ratio_value = data[ratio[0]]
            if ratio_value:
                ratio_values_list.append(ratio_value)
                ticker_list.append(ticker)
        sorted_tickers_list = sorted(ticker_list, key=lambda x: -ratio_values_list[ticker_list.index(x)])
        if not ratio[2]:
            sorted_tickers_list.reverse()
        # Only take the best 20 stocks for each ratio
        end_index = 20 if len(sorted_tickers_list) >= 20 else len(sorted_tickers_list)
        sorted_tickers_list = sorted_tickers_list[0:end_index]
        accepted_indexes = []
        for ind, val in enumerate(ticker_list):
            if val in sorted_tickers_list:
                accepted_indexes.append(ind)
        ticker_list = [ticker_list[i] for i in accepted_indexes]
        ratio_values_list = [ratio_values_list[i] for i in accepted_indexes]
        color_mapper = CategoricalColorMapper(factors=['CCJ'], palette=["green"], nan_color="blue")
        p = figure(width=width, height=height, x_range=sorted_tickers_list, title=f"{ratio[1]} TTM for Uranium Stocks")
        source = ColumnDataSource({"Ticker": ticker_list,
                                   ratio[1]: ratio_values_list})
        p.vbar(x="Ticker", top=ratio[1], width=0.9, fill_color={'field': 'Ticker', 'transform': color_mapper}, source=source)
        figures_list.append(p)
#     show(column(figures_list))
    return column(figures_list)
    
    
f = make_bar_charts_for_ratios([("quickRatioTTM", "Quick Ratio", True), 
                                    ('operatingProfitMarginTTM', "Operating Profit Margin", True), 
                                    ('returnOnEquityTTM', "Return On Equity", True),
                                    ('peRatioTTM', "Price to Earnings", False),
                                   ('debtEquityRatioTTM', 'Debt to Equity', False),
                                   ('dividendYielPercentageTTM', 'Dividend yield Percentage', True),
                                    ('enterpriseValueMultipleTTM', 'Enterprise Multiple', True)])
if not os.path.exists('plots'):
    os.mkdir('plots')
save(f, os.path.join(root, 'plots', "ttm_ratio_bar_charts.html"))

            

'/home/ben_luft/JupyterProjects/uraniumStocks/plots/ttm_ratio_bar_charts.html'

In [57]:
print(len(ratios_ttm_json))

66
